In [1]:
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.utils import coordinate_from_string, column_index_from_string

from openpyxl.styles import (PatternFill, 
                             Border, 
                             Side, 
                             Alignment, 
                             Protection, 
                             Font,
                             colors,
                             Color
                            )

%config IPCompleter.greedy=True 

In [2]:
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl import Workbook


def style_range(ws, cell_range, border=Border(), fill=None, font=None, alignment=None):
    """
    Apply styles to a range of cells as if they were a single cell.

    :param ws:  Excel worksheet instance
    :param range: An excel range to style (e.g. A1:F20)
    :param border: An openpyxl Border
    :param fill: An openpyxl PatternFill or GradientFill
    :param font: An openpyxl Font object
    """

    top = Border(top=border.top)
    left = Border(left=border.left)
    right = Border(right=border.right)
    bottom = Border(bottom=border.bottom)

    first_cell = ws[cell_range.split(":")[0]]
    if alignment:
#         ws.merge_cells(cell_range)
        first_cell.alignment = alignment

    rows = ws[cell_range]
    if font:
        first_cell.font = font

    for cell in rows[0]:
        cell.border = cell.border + top
    for cell in rows[-1]:
        cell.border = cell.border + bottom

    for row in rows:
        l = row[0]
        r = row[-1]
        l.border = l.border + left
        r.border = r.border + right
        if fill:
            for c in row:
                c.fill = fill


In [3]:
def getStringCoord (coord) :
    x,y = coord
    xl = get_column_letter(x)
    coord = "{0}{1}".format(xl, str(y))
    return coord

def c(icell,  add= None) :
    if add :
        icell = (icell[0] + add[0], icell[1] + add[1]
    return "{0}{1}".format(get_column_letter(icell[0]),icell[1])
    
c((2,34))

SyntaxError: invalid syntax (<ipython-input-3-7ee6f2e623d9>, line 10)

In [4]:
class TradeFormat :
    def __init__(self, a=(1,1) ):
        self.tradeAnchor = a

        #This first gtoup of ranges and styles refer to the larger font 3 by 2 merged sells --aka titleStyles
        
        # range1 is the amount to add. thus, with range = (2,1) the range of (1,1) goes to (3,2) covering a 3 by 2 grid
        self.range1 = (2,1)
        self.anchor1 = [
            (1,1), (4,1), (7,1), 
            (7,3), (7,5), (7, 7), 
            (7,9), (7,11), (7,13), (7,15), (7,17), (7,19), (7,21)
        ]
        self.titleStyles= (
            Font(color="FFFFFF", size=16),
            Alignment(horizontal="center", vertical="center", wrapText=True),
            PatternFill(start_color='A6A6A6', end_color='A6A6A6', fill_type='solid'),
            Border(left=Side(style='double'),
                   right=Side(style='double'), 
                   top=Side(style='double'), 
                   bottom=Side(style='double'))
        )

        # This second group of titles and fonts refer to the larger spaces (6 by 6 )  for notes
        self.range2=(5,5)
        self.anchor2 = [(1,11), (1,17)]

        self.noteStyles = (
            Font(color="000000", size=10),
            Alignment(horizontal="left", vertical="top", wrapText=True),
            PatternFill(start_color='FFFFCC', end_color='FFFFCC', fill_type='solid'),
            Border(left=Side(style='double'),
                   right=Side(style='double'), 
                   top=Side(style='double'), 
                   bottom=Side(style='double'))
            )
        self.range3= ("A3:F10")
        self.rangeNumFormat = ("B3:F8")
        self.normStyles = (
            Font(color="FFFFFF", size=11),
            Alignment(horizontal="left", vertical="bottom"),
            PatternFill(start_color='A6A6A6', end_color='A6A6A6', fill_type='solid'),
            Border(left=Side(style='double'),
                   right=Side(style='double'), 
                   top=Side(style='double'), 
                   bottom=Side(style='double'))
        )
        
        
        
    def mergeStuff(self, ws, anchor=(1,1), width=3, height=2) :
        width = width -1
        height = height -1
        ws.merge_cells(start_row=anchor[1], 
                       start_column=anchor[0], 
                       end_row=anchor[1]+height, 
                       end_column=anchor[0]+width)

        return (anchor, (anchor[0]+ width, anchor[1] + height))
    

    

    def formatTrade(self, ws, tstyle=None) :
        for r in self.anchor1 :
            cell = self.c(r)
            self.mergeStuff(ws, anchor = r)
#             print("something?", cell)
            
            ws[self.c(r)].font = self.titleStyles[0]
            ws[self.c(r)].alignment = self.titleStyles[1]
            ws[self.c(r)].fill = self.titleStyles[2]
            rng = "{0}:{1}".format(self.c(r), self.c(r, self.range1))
            print(rng)
            style_range(ws, rng, border=self.noteStyles[3])

            
        for r in self.anchor2 :
            self.mergeStuff(ws, anchor = r, width=6, height=6)
            
            ws[self.c(r)].font = self.noteStyles[0]
            ws[self.c(r)].alignment = self.noteStyles[1]
            ws[self.c(r)].fill = self.noteStyles[2]
            rng = "{0}:{1}".format(self.c(r), self.c(r, self.range2))
            print(rng)
# 
            style_range(ws, rng, border=self.noteStyles[3])
    
    
        first_cell = ws[self.range3.split(":")[0]]
        rows = ws[self.range3]
        for row in rows:
            for c in row:
                c.font = self.normStyles[0]
                c.alignment = self.normStyles[1]
                c.fill = self.normStyles[2]
                c.border = self.normStyles[3]
                
        rows = ws[self.rangeNumFormat]
        for row in rows:
            for c in row:
                c.number_format = '"$"#,##0.00_);[Red]\("$"#,##0.00\)'
                
    


        

        cc = ws['A1']
        ws['A1'] = 'Some trade short eh?'
        cc.value = 'Trade 4 -- MU Long'
        
        cc = ws['A1']
        ws['A1'] = 'Some trade short eh?'
        cc.value = 'Trade 4 -- MU Long'
        
        nn = ws[self.c((1,11))]
        nn.value = \
'''It was a long trade, it was a short trade it was a trade 
with too many words associated  with it it w as a trade with 
not enough said about it. All in all I am not saying anything 
new here  and in fact i am not saying anything at all i assure 
if you detect an y  meaning herea youo are mistaken'''

    def c(self, icell,  add= None) :
        if add :
            icell = (icell[0] + add[0], icell[1] + add[1])
        return "{0}{1}".format(get_column_letter(icell[0]),icell[1])
    
    def stupidFormat(self, ws) :
        for r in self.anchor1 :
            print(r)
            self.mergeStuff(ws, anchor = r)
            
    
   

In [5]:
tf = TradeFormat()
wb=Workbook()
ws = wb.active

In [6]:
tf.formatTrade(ws)

A1:C2
D1:F2
G1:I2
G3:I4
G5:I6
G7:I8
G9:I10
G11:I12
G13:I14
G15:I16
G17:I18
G19:I20
G21:I22
A11:F16
A17:F22


In [7]:
wb.save('out/justlookin.xlsx')

### 4 different currency formats:
"$"#,##0.00_);[Red]\("$"#,##0.00\)

"$"#,##0.00_);\("$"#,##0.00\)

"$"#,##0.00;[Red]"$"#,##0.00

"$"#,##0.00

In [ ]:
tf =TradeFormat()
tf.stupidFormat(ws)


In [ ]:

# try :
#     ws['A1'].style = '60 % - Accent3'
#     a3.style = '80 % - Accent3'
# exce?pt ValueError :



ws['A1'].fill = PatternFill(start_color='A6A6A6', end_color='A6A6A6', fill_type='solid')
ws['A1'].font = Font(color="FFFFFF", size=16)
ws['A1'].alignment = Alignment(horizontal="center", vertical="center")
        
        
                

### Some utility stuff i will need at some point

In [ ]:
xy = coordinate_from_string('A4') # returns ('A',4)
xy

In [ ]:
xy=('C45')
col = column_index_from_string(xy[0]) # returns 1
col

In [ ]:
xy[1]

In [ ]:

print(get_column_letter(1))

In [ ]:
def c(icell,  add= None) :
    if add :
        icell = (icell[0] + add[0], icell[1] + add[1])
    return "{0}{1}".format(get_column_letter(icell[0]),icell[1])

In [ ]:
a=(4,40)
b=(5,5)
D = (a[0] + b[0], a[1] + b[1]) 
D

In [ ]:
c(a,b)


In [ ]:
c(D)

### I think I will need this for borders